In [1]:
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Function to create the neural network model
def create_model(num_neurons, activation, learning_rate):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(num_neurons, activation=activation),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Defining the parameter space for the genetic algorithm
param_space = {
    'num_neurons': [32, 64, 128, 256],
    'activation': ['relu', 'sigmoid', 'tanh'],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Genetic Algorithm Parameters
population_size = 10
generations = 5
mutation_rate = 0.1

# Function to initialize the population
def initialize_population():
    population = []
    for _ in range(population_size):
        individual = {
            'num_neurons': np.random.choice(param_space['num_neurons']),
            'activation': np.random.choice(param_space['activation']),
            'learning_rate': np.random.choice(param_space['learning_rate'])
        }
        population.append(individual)
    return population

# Fitness function to evaluate an individual's performance
def fitness_function(individual):
    model = create_model(individual['num_neurons'], individual['activation'], individual['learning_rate'])
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, verbose=0)
    accuracy = history.history['val_accuracy'][-1]  # Last validation accuracy
    return accuracy

# Tournament selection function
def selection(population, fitness):
    selected = np.random.choice(population, size=2, replace=False)
    return max(selected, key=lambda x: fitness[str(x)])

# Crossover function to combine two parents
def crossover(parent1, parent2):
    child = {}
    for key in parent1.keys():
        child[key] = np.random.choice([parent1[key], parent2[key]])
    return child

# Mutation function to introduce random changes in individuals
def mutate(individual):
    if np.random.rand() < mutation_rate:
        individual['num_neurons'] = np.random.choice(param_space['num_neurons'])
    if np.random.rand() < mutation_rate:
        individual['activation'] = np.random.choice(param_space['activation'])
    if np.random.rand() < mutation_rate:
        individual['learning_rate'] = np.random.choice(param_space['learning_rate'])
    return individual

# Main Genetic Algorithm Loop
def genetic_algorithm():
    population = initialize_population()
    for generation in range(generations):
        print(f"Generation {generation + 1}/{generations}")
        fitness_scores = {str(individual): fitness_function(individual) for individual in population}
        new_population = []
        for _ in range(population_size // 2):
            parent1 = selection(population, fitness_scores)
            parent2 = selection(population, fitness_scores)
            child1 = mutate(crossover(parent1, parent2))
            child2 = mutate(crossover(parent1, parent2))
            new_population.extend([child1, child2])
        population = new_population
    best_individual = max(population, key=lambda x: fitness_function(x))
    return best_individual

# Run the genetic algorithm to find the best hyperparameters
best_params = genetic_algorithm()
best_params


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Generation 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Generation 2/5
Generation 3/5
Generation 4/5
Generation 5/5


{'num_neurons': 256, 'activation': 'relu', 'learning_rate': 0.001}